In [1]:
### Different approach - batch size learning 
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from fedavg import Fedavg
from sklearn.metrics import f1_score
from supported_modles import Supported_modles
import random
import utils

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [4]:
clients, text_x, test_y = utils.set_data_mock(selected_model, True)
f1_before = []
for client in clients:
    client.init_empty_model(selected_model)
    client.train_model()
    print(client.y.mean())
    y_hat = client.model.predict(text_x)
    f1_before.append(f1_score(test_y,y_hat, average="binary"))


Creating node1.
Creating node2.
Creating node3.
Creating node4.
0.9234279975450539


AttributeError: 'Net2nn' object has no attribute 'predict'

In [5]:
f1_before

[0.883544921875,
 0.8684833156563597,
 0.8673136552168487,
 0.8674737304726517,
 0.8722743944218291]

In [6]:
selected_model = Supported_modles.SGD_classifier

In [7]:
for client in clients:
    client.init_empty_model(selected_model,0.001)

In [8]:
fedavg = Fedavg("global", 0.001)
fedavg.init_global_model(selected_model, clients[0].model, 57)

[Errno 98] Address already in use
Waitiing for a Connection..


In [9]:
number_of_rounds = 50
batch_size = 0.2
epochs = 10
# prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(_,end=" ")

    applicable_clients = random.sample((clients), random.randint(1,4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size)
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [1]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = client.model.predict(text_x)
    f1_fedavg.append(f1_score(test_y,y_hat,average="binary"))
print(f1_fedavg)

NameError: name 'clients' is not defined

In [11]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference -0.007625912819148595
Difference -0.24145774327584713
Difference -0.09200208087347261
Difference -0.09975598029825883
Difference -0.028671781061678336
